In [1]:
from tqdm import tqdm
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
sys.path.append('/home/shenqi/Master_thesis')
import torch
from collections import defaultdict

from Neuronbench.utils.models import Vanilla, Vanilla_lif
from Neuronbench.utils.dataset import seq_dataloader

from metavision_ml.detection.anchors import Anchors
from metavision_ml.detection.rpn import BoxHead
from metavision_sdk_core import EventBbox

import snnmetrics.snnmetrics as sm

In [2]:
dataloader = seq_dataloader(num_tbins = 1, batch_size= 8)
model = Vanilla_lif(cin = dataloader.in_channels, cout = 256, base = 16)
box_coder = Anchors(num_levels=model.levels, anchor_list="PSEE_ANCHORS", variances=[0.1, 0.2])
head = BoxHead(model.cout, box_coder.num_anchors, len(dataloader.wanted_keys) + 1, 0)
model = model.to('cuda')
head = head.to('cuda')
model.load_state_dict(torch.load('../train_hybrid_spikingRED/save_models/14_model.pth',map_location=torch.device('cuda')))
head.load_state_dict(torch.load('../train_hybrid_spikingRED/save_models/14_pd.pth',map_location=torch.device('cuda')))


<All keys matched successfully>

In [3]:
synops = []
synops.append(sm.SynOps(fanout=torch.ones([256,45,80])))
synops.append(sm.SynOps(fanout=torch.ones([256,23,40])))
synops.append(sm.SynOps(fanout=torch.ones([256,12,20])))
synops.append(sm.SynOps(fanout=torch.ones([256,6,10])))
synops.append(sm.SynOps(fanout=torch.ones([256,3,5])))
batch_syn = [[] for _ in range(5)]


output_val_list = []
with tqdm(total=len(dataloader.seq_dataloader_test), desc=f'Testing',ncols=120) as pbar:
    for data in dataloader.seq_dataloader_test:
        pbar.update(1)
        inputs = data['inputs'].to(device='cuda')
        with torch.no_grad():
            feature = model(inputs)
            
            for i, feat in enumerate(feature):
                feature[i] = feature[i].to('cpu')
                batch_syn[i] = synops[i](feature[i])
       
    print(batch_syn[0]["synops_per_neuron"].mean())

/home/shenqi/Master_thesis/venv_torch1_13_cu117/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/shenqi/Master_thesis/venv_torch1_13_cu117/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
Testing: 100%|██████████████████████████████████████████████████████████████████████| 1200/1200 [01:06<00:00, 17.93it/s]

tensor(0.1169)
